In [2]:
start = 171
end = 172
count = 100
existing_pages = 2
extra_pages = 20

In [3]:
import yaml
import requests

# URL of the languages.yml file in the GitHub Linguist repository
url = "https://raw.githubusercontent.com/github/linguist/b3664e4f242c842e356bd011090f85514d147948/lib/linguist/languages.yml"

# Fetch the content of the YAML file
response = requests.get(url)
yaml_content = response.text

# Parse the YAML content
languages_data = yaml.safe_load(yaml_content)

# Extract language names
languages = list(languages_data.keys())

In [ ]:
import requests
import json

# GitHub API base URL
base_url = "https://api.github.com"

# GitHub personal access token
# Replace 'YOUR_ACCESS_TOKEN' with your actual token
access_token = 'YOUR_ACCESS_TOKEN'

# Function to fetch repository metadata for a given language
def fetch_repo_metadata(language, page):
    # Endpoint to search repositories by language
    search_endpoint = f"{base_url}/search/repositories"
    
    # Query parameters for the search endpoint
    params = {
        'q': f'language:{language}',
        'sort': 'stars',
        'order': 'desc',
        'per_page': 100,  # Number of results per page (maximum allowed)
        'page': page
    }
    
    # Headers including the access token
    headers = {
        'Authorization': f'token {access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    # Make the API request
    response = requests.get(search_endpoint, params=params, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Extract the first few repositories
        repositories = data['items'][:count]
        
        # Fetch language data for each repository and extract only required metadata
        repo_metadata = []
        for repo in repositories:
            # Fetch language data for the repository
            language_url = f"{base_url}/repos/{repo['full_name']}/languages"
            language_response = requests.get(language_url, headers=headers)
            if language_response.status_code == 200:
                # Parse language data
                languages = language_response.json()
                
                # Extract required metadata
                metadata = {
                    'name': repo['name'],
                    'description': repo['description'],
                    'languages': languages
                }
                
                repo_metadata.append(metadata)
            else:
                print(f"Failed to fetch language data for {repo['full_name']}. Status code: {language_response.status_code}")
        
        # Return metadata for each repository
        return repo_metadata
    else:
        # Print error message if request failed
        print(f"Failed to fetch repositories for {language}. Status code: {response.status_code}")
        return None

# Function to store repository metadata in a JSON file
def store_metadata(language, metadata):
    # File path to store metadata
    file_path = f"{language}_repos_metadata.json"
    
    # Write metadata to the JSON file
    with open(file_path, 'w') as file:
        json.dump(metadata, file, indent=4)
    
    print(f"Metadata for {language} repositories stored successfully in {file_path}")

def load_metadata(language):
    # File path for metadata
    file_path = f"{language}_repos_metadata.json"
    
    # Write metadata to the JSON file
    with open(file_path, 'r') as file:
        metadata = json.load(file)
        return metadata

# Iterate over each language
for i in range(start,end):
    language = languages[i]
    print(f"Fetching metadata for {language} repositories...")
        
    # Fetch repository metadata
    all_metadata = load_metadata(language)
    for p in range(existing_pages+1,extra_pages+existing_pages+1):
        repo_metadata = fetch_repo_metadata(language,p)
        if repo_metadata:
            all_metadata.extend(repo_metadata)
            store_metadata(language, all_metadata)
       
    print(f"Metadata retrieval for {language} complete.\n")


Fetching metadata for Go repositories...
Metadata for Go repositories stored successfully in Go_repos_metadata.json
Metadata for Go repositories stored successfully in Go_repos_metadata.json


In [72]:
import os
import json

merged_data = []

for file in json_files:
    file_path = os.path.join(current_dir, file)
    file_path = file_path + "_repos_metadata.json"
    with open(file_path, 'r') as f:
        try:
            json_data = json.load(f)
            merged_data.extend(json_data)
        except json.JSONDecodeError as e:
            print(f"Error reading file {file}: {e}")

with open("../all_repos.json", 'w') as f:
    json.dump(merged_data, f, indent=4)



In [22]:
with open("../langList.json", 'r') as f:
    try:
        json_files = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error reading file {file}: {e}")

In [34]:
with open("../urgent.txt", 'w') as f:
    f.write("language: repos, aprox index\n")
    for i in range(0,511):
        r = merged_data[i]
        if (len(r) < 200):
            f.write(f"{json_files[i]}: {len(r)} {i}\n")

In [69]:
len(merged_data)

80384

In [119]:
languages[495]

'TypeScript'

In [71]:
# Read data from langList.json
with open('../langList.json', 'r') as file:
    lang_list = json.load(file)

# Create a dictionary mapping string names to their index
lang_dict = {lang: index for index, lang in enumerate(lang_list)}

# Write the dictionary to a JSON file
with open('../list2lang.json', 'w') as file:
    json.dump(lang_dict, file, indent=4)